In [2]:
token = "lip_gPT8yQQYemHXfBNX2wwP" # here should be lichess API token, but i will hide my token because its confidential data

In [3]:
import pandas as pd
import numpy as np
import berserk
import json

In [4]:
session = berserk.TokenSession(token)
client = berserk.Client(session=session)

In [5]:
duno = client.games.export_by_player('EvilSaintPL', max=500) # posible adding max=300, because without limit it takes some time

In [6]:
ranking_data = client.users.get_rating_history('EvilSaintPL')

In [7]:
df1 = pd.read_json(json.dumps(list(duno), default=str))
df = df1.copy()

In [8]:
df = df1.copy()

In [9]:
df.head(1)
# at this point im looing into data, and deciding what data do i need
# in this case i dont need moves, only a amount of moves played by side
# also its not important if was a game played in tourament
# days per turn have value only for corespondic chess games so i can drop it too
# swiss and initialFen also are not important
# from players -> user i need only one result, so i will take name + will add ranking points to players
# ratingDiff will be calculated later on my own

,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,players,winner,moves,clock,tournament,initialFen
0,zGIqZ3sO,True,standard,blitz,blitz,2023-07-01 17:31:52.480000+00:00,2023-07-01 17:41:22.575000+00:00,resign,"{'white': {'user': {'name': 'Davidc323', 'patr...",black,d4 d5 e3 e5 dxe5 Nc6 Nf3 Bg4 Be2 Nge7 Nbd2 g6 ...,"{'initial': 300, 'increment': 0, 'totalTime': ...",NaN,NaN


In [14]:
df.isna().sum()

id              0
rated           0
variant         0
speed           0
perf            0
createdAt       0
lastMoveAt      0
status          0
players         0
winner         27
moves           0
clock           0
tournament    494
initialFen    497
dtype: int64

# dealing with moves, tournament and players columns

In [30]:
df["movesperplayer"] = df["moves"].str.count(" ")//2 # changing moves column to numer of moves instead of notation
df.drop("tournament", axis=1, inplace=True) # droping unnessesary column
df.drop("initialFen", axis=1, inplace=True)
df.drop("swiss", axis=1, inplace=True)
df.drop("daysPerTurn", axis=1, inplace=True)

In [32]:
df.iloc[0,7]

'outoftime'

In [33]:
def player(x: dict):
    if ('white' in x.keys()) & ('black' in x.keys()):
        if ("user" in x["white"].keys()) & ("user" in x["black"].keys()):
            plyer_data = {
                "white_player_name" : x['white']['user']['name'],
                "white_ranking" : x['white']['rating'],
                "black_player_name" : x['black']['user']['name'],
                "black_ranking" : x['black']['rating']
            }
        elif ("user" in x["white"].keys()):
            plyer_data = {
                "white_player_name" : x['white']['user']['name'],
                "white_ranking" : x['white']['rating'],
               "black_player_name" : "unknown",
                "black_ranking" : 0
            }
        elif ("user" in x["black"].keys()):
            plyer_data = {
                "black_player_name" : x['black']['user']['name'],
                "black_ranking" : x['black']['rating'],
                "white_player_name" : "unknown",
                "white_ranking" : 0
            }
    else:
        plyer_data = {"white_player_name":"unknown","white_ranking":0,"black_player_name":unknown,"black_ranking":0}
    result = pd.DataFrame(plyer_data, index=[0])
    return result


In [34]:
df.iloc[0,7]

'outoftime'

In [35]:
c = ["white_player_name","white_ranking","black_player_name","black_ranking"]
finit = df["players"].apply(player)
result=pd.DataFrame(columns=c)
for i in range(len(finit)):
    result = pd.concat([result, finit[i]])
result =result.reset_index().drop("index", axis=1)
df = pd.concat([df, result], axis=1)  # adding separated players columns 
df.drop("players", axis=1, inplace=True) # droping players column

In [36]:
df.head(2)

,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking
0,O83cQxfo,True,standard,blitz,blitz,2023-06-28 00:33:43.673000+00:00,2023-06-28 00:44:00.722000+00:00,outoftime,white,e4 e5 Nf3 Nc6 Bc4 Nf6 d4 exd4 e5 d5 exd6 Qxd6 ...,"{'initial': 300, 'increment': 0, 'totalTime': ...",70,EvilSaintPL,1753,Leosammie,1735
1,KeBcqcHz,True,standard,blitz,blitz,2023-06-28 00:31:45.081000+00:00,2023-06-28 00:33:30.178000+00:00,resign,white,Nf3 Nc6 g3 e5 Bg2 e4 Ng1 d5 d3 f5 dxe4 fxe4 Nc...,"{'initial': 300, 'increment': 0, 'totalTime': ...",18,convict-19,1801,EvilSaintPL,1758


# dealing with clock and perf column

In [37]:
df.iloc[0,10]

{'initial': 300, 'increment': 0, 'totalTime': 300}

In [38]:
def clock(x):
    if type(x)==dict:
        if 'initial' in x.keys():
            return x["initial"]
    else:
        return 0

In [39]:
df["clock"]=df["clock"].apply(clock)

# deling with time columns and simple cleaning

In [40]:
df["createdAt"] = df["createdAt"].apply(lambda x: pd.to_datetime(x).tz_convert('Europe/Berlin'))
df["lastMoveAt"] = df["lastMoveAt"].apply(lambda x: pd.to_datetime(x).tz_convert('Europe/Berlin'))

In [41]:
df.head(2)

,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking
0,O83cQxfo,True,standard,blitz,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,e4 e5 Nf3 Nc6 Bc4 Nf6 d4 exd4 e5 d5 exd6 Qxd6 ...,300,70,EvilSaintPL,1753,Leosammie,1735
1,KeBcqcHz,True,standard,blitz,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,Nf3 Nc6 g3 e5 Bg2 e4 Ng1 d5 d3 f5 dxe4 fxe4 Nc...,300,18,convict-19,1801,EvilSaintPL,1758


In [42]:
df.status[df.status=='outoftime'].count()

802

In [43]:
df.drop(df[df.variant!="standard"].index, inplace=True)

In [44]:
df.drop(df[df.speed=="correspondence"].index, inplace=True)
df.drop(df[df.speed=='ultraBullet'].index, inplace=True)

In [45]:
df.drop("perf", axis=1, inplace=True)

In [46]:
df.drop(df[df.status=="noStart"].index, inplace=True)

In [47]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,e4 e5 Nf3 Nc6 Bc4 Nf6 d4 exd4 e5 d5 exd6 Qxd6 ...,300,70,EvilSaintPL,1753,Leosammie,1735
1,KeBcqcHz,True,standard,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,Nf3 Nc6 g3 e5 Bg2 e4 Ng1 d5 d3 f5 dxe4 fxe4 Nc...,300,18,convict-19,1801,EvilSaintPL,1758


In [48]:
df.to_csv('Chess_data_basic.csv')